In [22]:
import pandas as pd
import datetime as dt

In [97]:
#get ticker list of all 8218 stocks
import glob
file_name_list = glob.glob(r"C:\Users\Carso\Desktop\Eldon\stock_price\*.txt") #Tiingo stock price data folder
ticker_list = []
for file in file_name_list:
    ticker = file.replace("C:\\Users\\Carso\\Desktop\\Eldon\\stock_price\\",'').replace('.txt','')
    ticker_list.append(ticker)

In [188]:
#filter out stocks that has no price data
#less than 54 days of price data. 
filtered_ticker_list = []
for ticker in ticker_list:
    if pd.read_csv(r"C:\Users\Carso\Desktop\Eldon\stock_price\%s.txt"%ticker).shape[0] >= 1:
        filtered_ticker_list.append(ticker)
len(filtered_ticker_list)

In [251]:
#calculate daily average of popularity data, store them in a dictionary called "daily_popularity_avg"
def calc_daily_popularity_avg(ticker):
    popularity = pd.read_csv(r"C:\Users\Carso\Desktop\Eldon\213411_951860_bundle_archive\tmp\popularity_export\%s.csv"%ticker)#robintrack popularity data folder
    popularity['date'] = popularity.timestamp.apply(lambda x:x[:10])
    popularity_avg = popularity.groupby(by = "date")['users_holding'].mean().to_frame()
    return popularity_avg
daily_popularity_avg = {}
for ticker in filtered_ticker_list:
    popularity_avg = calc_daily_popularity_avg(ticker)
    daily_popularity_avg[ticker] = popularity_avg

In [252]:
#get stock price data using ticker
def get_price_data(ticker):
    return pd.read_csv(r"C:\Users\Carso\Desktop\Eldon\stock_price\%s.txt"%ticker).loc[:,['date',"close"]]

In [275]:
#join stock price data and daily popularity average data using "date"
#store the combined dataset in a dictionary called "combined_data"
def join_price_pop(ticker):
    price_data = get_price_data(ticker)
    pop_data = daily_popularity_avg[ticker]
    return price_data.merge(pop_data, how = "inner", left_on = "date", right_on = "date")
combined_data = {}
for ticker in filtered_ticker_list:
    combined_data[ticker] = join_price_pop(ticker)

In [383]:
#filter out stocks that have less than 54 days of combined data, call the new dict "filtered_combined_data"
#this is to make sure all stocks we use have at least 10 complete weeks of combined data (each week has 5 trading days)
filtered_ticker_list_2 = []
for ticker in filtered_ticker_list:
    if combined_data[ticker].shape[0] >= 54:
        filtered_ticker_list_2.append(ticker)
filtered_combined_data = {}
for ticker in filtered_ticker_list_2:
    filtered_combined_data[ticker] = combined_data[ticker]
len(filtered_ticker_list_2)

7935

In [384]:
#label day of the week for each stock in combined_data
for ticker in filtered_ticker_list_2:
    filtered_combined_data[ticker]['day_of_week'] = filtered_combined_data[ticker]['date'].apply(lambda x: dt.datetime.strptime(x, "%Y-%m-%d").weekday())

In [388]:
#find index of first friday
def get_first_friday(ticker):
    if filtered_combined_data[ticker].loc[0,'day_of_week'] == 4:
        index = 0
    else:
        index = 4-filtered_combined_data[ticker].loc[0,'day_of_week']
    return index

In [389]:
#find index of last friday
def get_last_friday(ticker):
    if filtered_combined_data[ticker].loc[(filtered_combined_data[ticker].shape[0]-1),'day_of_week']==4:
        index = filtered_combined_data[ticker].shape[0]-1
    else:
        index = filtered_combined_data[ticker].shape[0]- filtered_combined_data[ticker].loc[(filtered_combined_data[ticker].shape[0]-1),'day_of_week'] - 4
    return index

In [390]:
#cut off days before first friday and after last friday, and label week numbers between them
def label_week_no(ticker, first_friday_index, last_friday_index):
    ticker_cutted = filtered_combined_data[ticker][first_friday_index:(last_friday_index+1)].reset_index(drop = True)
    week_count = 0
    for i in range(ticker_cutted.shape[0]):
        ticker_cutted.loc[i,'week_number'] = week_count
        if ticker_cutted.loc[i,'day_of_week'] == 4:
            week_count += 1
    return ticker_cutted
for ticker in filtered_ticker_list_2:
    first_friday  = get_first_friday(ticker)
    last_friday = get_last_friday(ticker)
    filtered_combined_data[ticker] = label_week_no(ticker, first_friday, last_friday)

In [391]:
#calculate the stock price return
def get_return (ticker):
    for i in range(1,filtered_combined_data[ticker].shape[0]):
        filtered_combined_data[ticker].loc[i,'return'] = filtered_combined_data[ticker].loc[i,'close']/filtered_combined_data[ticker].loc[i-1, 'close'] - 1
    return filtered_combined_data[ticker].loc[1:,:]
for ticker in filtered_ticker_list_2:
    filtered_combined_data[ticker] = get_return(ticker)

In [396]:
#calculate standard deviation of weekly stock return, and weekly average popularity, store them into new dict "output", then output to csv
import numpy as np
output = {}
for ticker in filtered_ticker_list_2:
    weekly_std = filtered_combined_data[ticker].groupby(by = "week_number").agg({"return":np.std})
    weekly_avg_pop = filtered_combined_data[ticker].groupby(by = "week_number")['users_holding'].mean().to_frame()
    output[ticker] = weekly_std.merge(weekly_avg_pop, how = "inner", left_on = "week_number", right_on = "week_number")
    output[ticker] = output[ticker].rename(columns = {"return":"std"})
    output[ticker].to_csv(r"C:\Users\Carso\Desktop\Eldon\output\%s.txt"%ticker, index=True)# output folder

In [398]:
output["ZYXI"]

,std,users_holding
week_number,,
1.0,0.012866,53.218750
2.0,0.041433,99.641667
3.0,0.057974,104.425000
4.0,0.033376,121.875000
5.0,0.009899,166.208333
6.0,0.012433,201.366667
7.0,0.018806,218.816667
8.0,0.047816,240.493590
9.0,0.008856,288.736111
